In [4]:
import spacy

import torch
from torch import nn
import torch.nn.functional as F

from torchtext.datasets import TranslationDataset
from torchtext.data import Field, BucketIterator, TabularDataset
from torch import optim

import time
import random
from helper.helper import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:

ARTICLE = Field(tokenize='spacy',
                tokenizer_language='en',
                init_token='<sos>',
                eos_token='<eos>',
                lower=True,
                include_lengths=True
                )
SUMMARY = Field(tokenize='spacy',
                tokenizer_language='en',
                init_token='<sos>',
                eos_token='<eos>',
                lower=True,
                )

In [ ]:
train_data, valid_data, test_data = TabularDataset.splits(
                                                    path="./data",
                                                    train='train.csv',
                                                    validation='val.csv',
                                                    test='test.csv', 
                                                    format='csv',
                                                    fields=[("text",ARTICLE),('headline',SUMMARY)]
                                                        )

In [ ]:
ARTICLE.build_vocab(train_data, min_freq=2)
SUMMARY.build_vocab(train_data, min_freq=2)

In [ ]:
with open("./model/ARTICLE.Field","wb")as f:
     dill.dump(ARTICLE,f)
with open("./model/SUMMARY.Field","wb")as f:
     dill.dump(SUMMARY,f)

In [ ]:
BATCH_SIZE = 64

train_loader, valid_loader, test_loader = BucketIterator.splits(
                                                            (train_data, valid_data, test_data),
                                                            batch_size=BATCH_SIZE,
                                                            sort_within_batch=True,
                                                            sort_key = lambda x:len(x.text),
                                                            device=device
                                                                )

In [ ]:
class Encoder(nn.Module):
    
    
    def __init__(self, vocab, embeding_dim, encoder_hidden_dim, decoder_hidden_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab, embeding_dim)
        self.rnn = nn.GRU(embeding_dim, encoder_hidden_dim, bidirectional = True)
        self.fc = nn.Linear(encoder_hidden_dim*2, decoder_hidden_dim)
        
        self.dropout = nn.Dropout(p=dropout)
   

    def forward(self, text, text_len):
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_len)
        packed_outputs, hidden = self.rnn(packed_embedded) 
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs)
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        return outputs, hidden

In [ ]:
class Attention(nn.Module):
    
    
    def __init__(self, enc_hid_dim, dec_hid_dim ):
        super().__init__()
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
    
    
    def forward(self, hidden, encoder_outputs, mask):      
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        attention = self.v(energy).squeeze(2)
        attention = attention.masked_fill(mask == 0, -1e10)
        
        return F.softmax(attention, dim = 1)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()
        
        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)        
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input_, hidden, encoder_outputs, mask):
    
        input_ = input_.unsqueeze(0)
        embedded = self.dropout(self.embedding(input_))

        atten = self.attention(hidden, encoder_outputs, mask)        
        atten = atten.unsqueeze(1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        weighted = torch.bmm(atten, encoder_outputs)
        
        weighted = weighted.permute(1, 0, 2)
        
        rnn_input_ = torch.cat((embedded, weighted), dim = 2)

            
        output, hidden = self.rnn(rnn_input_, hidden.unsqueeze(0))
        
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        return prediction, hidden.squeeze(0), atten.squeeze(1)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, text_pad_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.text_pad_idx = text_pad_idx
        self.device = device
        
    def create_mask(self, text):
        mask = (text != self.text_pad_idx).permute(1, 0)
        return mask
        
    def forward(self, text, text_len, headline, teacher_forcing_ratio = 0.5):
        batch_size = text.shape[1]
        headline_len = headline.shape[0]
        headline_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(headline_len, batch_size, headline_vocab_size).to(self.device)
      
        encoder_outputs, hidden = self.encoder(text, text_len)
       
        input_ = headline[0,:]
        
        mask = self.create_mask(text)
                
        for t in range(1, headline_len):
            
            output, hidden, _ = self.decoder(input_, hidden, encoder_outputs, mask)           
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio            
            top1 = output.argmax(1)             
            input_ = headline[t] if teacher_force else top1
            
        return outputs

In [ ]:
with open("ARTICLE.Field","rb")as f:
     ARTICLE=dill.load(f)
with open("SUMMARY.Field","rb")as f:
     SUMMARY=dill.load(f)

In [ ]:
INPUT_DIM = len(ARTICLE.vocab)
OUTPUT_DIM = len(SUMMARY.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
ARTICLE_PAD_IDX = ARTICLE.vocab.stoi[ARTICLE.pad_token]

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, ARTICLE_PAD_IDX, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
SUMMARY_PAD_IDX = SUMMARY.vocab.stoi[SUMMARY.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = SUMMARY_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        text, text_len = batch.text
        headline = batch.headline
        
        optimizer.zero_grad()
        
        output = model(text, text_len, headline)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        headline = headline[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, headline)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            text, text_len = batch.text
            headline = batch.headline

            output = model(text, text_len, headline, 0) #turn off teacher forcing
            
            #headline = [headline len, batch size]
            #output = [headline len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            headline = headline[1:].view(-1)

            #headline = [(headline len - 1) * batch size]
            #output = [(headline len - 1) * batch size, output dim]

            loss = criterion(output, headline)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_loader, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.brain')
    
    print(f'Epoch: {epoch+1:2} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

In [ ]:
model.load_state_dict(torch.load('model.brain', map_location=device))

test_loss = evaluate(model, test_loader, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

In [ ]:
def predict(sentence, src_field, trg_field, model, device, max_len = 50):

    model.eval()
        
    if sentence is str:
        nlp = spacy.load('en')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)
    src_len = torch.LongTensor([len(src_indexes)]).to(device)
    
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor, src_len)

    mask = model.create_mask(src_tensor)        
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]
    attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)
    
    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
        
        with torch.no_grad():
            output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs, mask)

        attentions[i] = attention            
        pred_token = output.argmax(1).item()        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attentions[:len(trg_tokens)-1]